In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from feature_selectors import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split

In [ ]:
gebruikers_df = pd.read_csv(r'..\..\data\gebruikersinformatie.csv', delimiter = ';')
evenement_df = pd.read_csv(r'..\..\data\evenementinformatie.csv', delimiter = ';')
aanwezigheid_df = pd.read_csv(r'..\..\data\evenementaanwezigheid.csv', delimiter = ';')

In [ ]:
df = merge_on_aanwezigheid(aanwezigheid_df, evenement_df, gebruikers_df)

df

In [ ]:
# Remove nonvalid entries
for col in df.columns:
    df = df[~(df[col].isna())]

df = df[~(df['Gebruiker_Lidmaatschapstype'] == 'x')]
df

In [ ]:
# Convert ID to ints
to_convert = ["EvenementID", "GebruikerID", "Evenement_OrganisatorID"]

for col in to_convert:
    df[col] = df[col].str[1:].astype(int)

df

In [ ]:
# Factorize
to_factorize = ["Aanwezigheidsstatus", "Gebruiker_Lidmaatschapstype"]

for col in to_factorize:
    df[col], unique_values = pd.factorize(df[col])

    print(f"\n{col} numerics:")
    for i, value in enumerate(unique_values):
        print(f"{i} -> {value}")


df['Gebruiker_Lidmaatschapstype'] = df['Gebruiker_Lidmaatschapstype'].astype(bool)

# Factorize studiejaar with map
df["Gebruiker_Studiejaar"] = df["Gebruiker_Studiejaar"].map({
    "eerstejaars": 1,
    "tweedejaars": 2,
    "derdejaars": 3,
    "vierdejaars": 4
})

# Aanwezigheidsstatus (0 = aanwezig, 1 = afwezig)
df['Aanwezigheidsstatus'] = df['Aanwezigheidsstatus'].replace(2, 1).astype('bool', )

df

In [ ]:
# Dummies from evenemnttype and studierichting
df = pd.concat([df, pd.get_dummies(df['Evenement_EvenementType'], prefix = 'Evenement_EvenementType')], axis = 1)
df = pd.concat([df, pd.get_dummies(df['Gebruiker_Studierichting'], prefix = 'Gebruiker_Studierichting')], axis = 1)
df.drop(columns = ['Evenement_EvenementType', 'Gebruiker_Studierichting'], axis = 1, inplace = True)

df

In [ ]:
# Convert columns to datetime datatype
date_columns = ['Evenement_EvenementDatum', 'Gebruiker_RegistratieDatum', 'Gebruiker_LaatsteLogin']

for col in date_columns:
    df[col] = pd.to_datetime(df[col], dayfirst = True)

# Extract date/month/year from datetime columns
for col in date_columns:
    df[f'{col}_is_weekend'] = df[col].dt.weekday >= 5
    df[f'{col}_month'] = df[col].dt.month
    # df[f'{col}_year'] = df[col].dt.year

df.drop(columns = date_columns, inplace = True)

df

In [ ]:
def get_all_predictors():
    all_predictors = df.columns.tolist()
    all_predictors.remove('EvenementID')
    all_predictors.remove('GebruikerID')
    all_predictors.remove('Evenement_OrganisatorID')

    return all_predictors

df

In [ ]:
x = sub_df[all_predictors].drop(columns=[col for col in sub_df.columns if col.startswith('Gebruiker_')], axis=1)
x.drop('Aanwezigheidsstatus', axis = 1, inplace = True)
y = sub_df['Aanwezigheidsstatus']

In [ ]:
states = 100
depths = 15
best_depth_frequency = {}

# For each randomstate
for state in range(states):
    train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.6, shuffle = True, random_state = state)
    results = {}

    # Save score on each depth
    for depth in range(1, depths):
        model = DecisionTreeClassifier(max_depth = depth).fit(train_x, train_y)
        results[depth] = accuracy_score(test_y, model.predict(test_x))

    # Determine depth with the highest score
    best_depth = max(results, key = results.get)

    # Add a frequency point to that depth
    best_depth_frequency[best_depth] = best_depth_frequency.get(best_depth, 0) + 1

# Determine most frequent best depth
most_frequent_best_depth = max(best_depth_frequency, key = best_depth_frequency.get)

# Plot results
print(f'Most frequent best depth: {most_frequent_best_depth}')
plt.bar(best_depth_frequency.keys(), best_depth_frequency.values())
plt.xlabel('Depth')
plt.ylabel('Frequency')
plt.show()

# Train model one last time with most frequent best depth
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.6, shuffle = True, random_state = 0)
model = DecisionTreeClassifier(max_depth = most_frequent_best_depth).fit(train_x, train_y)

In [ ]:
model = DecisionTreeClassifier(random_state=1, max_depth=most_frequent_best_depth)
user_id = 1
all_predictors = get_all_predictors()
sub_df = df[df['GebruikerID'] == user_id]

X = sub_df[all_predictors].drop(columns=[col for col in sub_df.columns if col.startswith('Gebruiker_')], axis=1)
X.drop('Aanwezigheidsstatus', axis = 1, inplace = True)
Y = sub_df['Aanwezigheidsstatus']

tree = model.fit(X, Y)
print(tree.classes_)
plot_tree(tree, feature_names = X.columns, class_names = list(map(str, tree.classes_)), fontsize = 7)